In [40]:
import pandas as pd
import numpy as np
import config

In [41]:
import os
os.environ['CHILDES_EC2_PASS'] = config.CHILDES_EC2_PASS 
os.environ['ROOT_PASS'] = config.ROOT_PASS 

In [3]:
import compare_refactor

In [42]:
local_params = {
    "host": "127.0.0.1",
    "current":"childes_db_dev",
        "supported": ["childes_db_dev", "childes_db_dev_test2","2020.1"],
        "historical":[],
    "user": "root",
    "password": os.environ['ROOT_PASS']
}

In [43]:
expected_name = "2020.1" # the previously obtained values
actual_name = "childes_db_dev_test2" # the new values

expected_con = compare_refactor.connect_to_childes(db_version = expected_name, db_args = local_params)
actual_con = compare_refactor.connect_to_childes(db_version = actual_name, db_args = local_params)

In [6]:
# transcript_id from expected
# transcript_id from actual
# 1881,1878

expected_transcript_id = 1881
actual_transcript_id = 1878 

### Replicate the hash mismatch

In [25]:
import imp
imp.reload(compare_refactor)

<module 'compare_refactor' from '/home/stephan/notebooks/childes-db/compare_refactor.py'>

<module 'compare_refactor' from '/home/stephan/notebooks/childes-db/compare_refactor.py'>

In [15]:
compare_refactor.compare_hashes(actual_transcript_id, expected_transcript_id, actual_con, expected_con, query = "SELECT * FROM token WHERE transcript_id = %s")

,actual,expected
0,11752571090508953572,7187927096151430765
1,11816686989527058195,2320030376840308126
2,15086620079007162987,2079552189445050214
3,286531674265746032,10735321540557915689
4,16910328023804432037,12270852186046702956
...,...,...
286,7766737628147443465,5356293645191094960
287,9625482461141638974,5812636098772289651
288,8374366981155887588,15844656010636851297
289,13246468752038423764,12278467948386166833


,actual,expected
0,11752571090508953572,7187927096151430765
1,11816686989527058195,2320030376840308126
2,15086620079007162987,2079552189445050214
3,286531674265746032,10735321540557915689
4,16910328023804432037,12270852186046702956
...,...,...
286,7766737628147443465,5356293645191094960
287,9625482461141638974,5812636098772289651
288,8374366981155887588,15844656010636851297
289,13246468752038423764,12278467948386166833


### Find the contents that are different

In [16]:
actual_transcript_id = 1879
expected_transcript_id = 1880
compare_refactor.compare_dfs(actual_transcript_id, expected_transcript_id, actual_con, expected_con, query = "SELECT * FROM token WHERE transcript_id = %s")


target_child_sex      
                self other
0             female  male
1             female  male
2             female  male
3             female  male
4             female  male
..               ...   ...
286           female  male
287           female  male
288           female  male
289           female  male
290           female  male

[291 rows x 2 columns]

target_child_sex      
                self other
0             female  male
1             female  male
2             female  male
3             female  male
4             female  male
..               ...   ...
286           female  male
287           female  male
288           female  male
289           female  male
290           female  male

[291 rows x 2 columns]

### Run all discrepanices from the log

In [19]:
token_errors = pd.read_csv('/home/snair/childes-db/token_errors.log')
token_errors.head(5)

,main_id,refactor_id,local_fname,collection,empty_for_trnst
0,1881,1878,Frogs/English-MiamiMono/mleng2/31321413.xml,Frogs,neither
1,1880,1879,Frogs/English-MiamiMono/mleng2/31321414.xml,Frogs,neither
2,1876,1880,Frogs/English-MiamiMono/mleng2/31321319.xml,Frogs,neither
3,1879,1881,Frogs/English-MiamiMono/mleng2/31321347.xml,Frogs,neither
4,1882,1882,Frogs/English-MiamiMono/mleng2/31321445.xml,Frogs,neither


,main_id,refactor_id,local_fname,collection,empty_for_trnst
0,1881,1878,Frogs/English-MiamiMono/mleng2/31321413.xml,Frogs,neither
1,1880,1879,Frogs/English-MiamiMono/mleng2/31321414.xml,Frogs,neither
2,1876,1880,Frogs/English-MiamiMono/mleng2/31321319.xml,Frogs,neither
3,1879,1881,Frogs/English-MiamiMono/mleng2/31321347.xml,Frogs,neither
4,1882,1882,Frogs/English-MiamiMono/mleng2/31321445.xml,Frogs,neither


In [33]:
token_error_results = []
for transcript_row in token_errors.to_dict('records'):
    actual_transcript_id = transcript_row['refactor_id']
    expected_transcript_id = transcript_row['main_id']
    col_comparison = compare_refactor.compare_dfs(actual_transcript_id, expected_transcript_id, actual_con, expected_con, query = "SELECT * FROM token WHERE transcript_id = %s",
                                return_type = 'colnames')
    
    token_error_results.append(' '.join(col_comparison))    
token_error_results

['target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_chil

['target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_chil

In [34]:
token_errors_augmented = token_errors
token_errors['mismatched_columns'] = token_error_results
token_errors.to_csv('token_errors_augmented.log')


In [36]:
pd.Series(token_error_results).value_counts()

target_child_sex                                   1573
speaker_name target_child_name                      125
speaker_name target_child_name target_child_sex      66
speaker_name                                         45
dtype: int64

target_child_sex                                   1573
speaker_name target_child_name                      125
speaker_name target_child_name target_child_sex      66
speaker_name                                         45
dtype: int64

In [ ]:
# [X] make a "compare hashes command"
# [X] Make a "compare dataframes" function that looks for differences programatically within the dataframe
# [X] See if gender is the common problem for other ones -- some of them at least
# [X] Move functions into the compare_refactor.py code
# [X] run this on the log output to see if we can identifiy what went wrong in each case